# <center>Tumeur Cérébrale

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from flask import Flask, render_template, request, url_for, redirect,jsonify,session
from werkzeug.utils import secure_filename
import json
import os
import pandas as pd
import seaborn as sns
import zipfile
import sys
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import glob
import random
%matplotlib inline
import warnings #
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.morphology import label
import tensorflow as tf

**Fonctions dont nous avons besoin**

Redimensionnement des images

In [ ]:
def redim(im): 
    img_resized = cv2.resize(im,(120,120))
    return img_resized

Transformation en niveau de gris

In [ ]:
def rgb2gray(I):
    return np.dot(I,[0.2989, 0.5870, 0.1140])

Calcul d'histogramme

In [ ]:
def histo(I):
    nl,nc=np.shape(I)
    h=np.zeros(256)
    I_m=np.round(I)
    for i in range(0,nl):
        for j in range(0,nc):
            val=int(I_m[i][j])
            h[val]+=1
    return h

Calcul la forme

In [ ]:
def Laplacian(I):
    img_lap=cv2.Laplacian(I,cv2.CV_8U,-1)
    r, g, b = img_lap[:,:,0], img_lap[:,:,1], img_lap[:,:,2]
    gray_lap= 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray_lap

Distance Euclidienne

In [ ]:
def distance(x,y):
    d=np.sqrt(np.sum(np.power((x-y),2)))
    return d

Distance de Manhattan

In [ ]:
def distanceMan(x,y):
    d=np.sum(np.abs(x-y))
    return d

Chemin pour base de données

In [ ]:
#Training Paths:
pathn=r'Dataset\Data1\No\\'
pathy=r'Dataset\Data1\Yes\\'

#Testing Paths:
testpathn=r'Dataset\Data1\Testing\No\\'
testpathb=r'Dataset\Data1\Testing\Benign\\'
testpathm=r'Dataset\Data1\Testing\Malignant\\'

Calcul des statistiques d'ordre 1

In [ ]:
def variance(I):
    nl,nc=I.shape
    m=np.mean(I)
    va=np.sum((I-m)**2)/(nl*nc)
    return va

In [ ]:
def energie(I): 
    nl,nc=I.shape
    en= np.sum(I**2)
    return en

In [ ]:
def entropie(I):
    ent=np.sum(I*np.log2(I+1e-10))
    return -ent

In [ ]:
def contraste(I):
    nl,nc=I.shape
    co=0
    for i in range(nl):
        for j in range(nc):
            co+=np.power((i-j),2)*I[i,j]
    return co

In [ ]:
def homogenite(I):
    nl,nc=I.shape
    mo=0
    for i in range(nl):
        for j in range(nc):
            mo+=I[i,j]/(1+np.abs(i-j))
    return mo

Stockage des 5 statistiques dans un array

In [ ]:
def texture(I):
    I=I/255
    return np.array((variance(I),energie(I),entropie(I),contraste(I),homogenite(I)))

Calcul de la matrice de co-occurence

In [ ]:
def GLCM(I):
    (l,c)=I.shape
    I=np.array(I).astype(int)
    m=np.max(I)
    C=np.zeros((m+1,m+1));
    for i in range(l):
        for j in range(c-1):
            C[I[i][j]][I[i][j+1]]+=1
    return C/np.max(C)

Stockage des images dans une matrice train

In [ ]:
def stocker_images_train(nbr_im):
    filename1=pathn+'N1.jpg'
    I=plt.imread(filename1)
    I=redim(I)
    nl,nc=rgb2gray(I).shape
    mat_image=np.zeros((nl,nc,nbr_im))
    
    for i in range(nbr_im):
        if(i<nbr_im/3):
            I2=plt.imread(pathn+'N'+str(i+1)+'.jpg')
            I2=redim(I2)
            mat_image[:,:,i]=np.round(rgb2gray(I2))
        elif(i<2*nbr_im/3):
            I2=plt.imread(pathy+'B'+str(i-int(nbr_im/3)+1)+'.jpg')
            I2=redim(I2)
            mat_image[:,:,i]=np.round(rgb2gray(I2))
        elif(i<nbr_im):
            I2=plt.imread(pathy+'M'+str(i-int(2*nbr_im/3)+1)+'.jpg')
            I2=redim(I2)
            mat_image[:,:,i]=np.round(rgb2gray(I2))
    
    return mat_image

def stocker_images_train_form(nbr_im):
    filename1=pathn+'N1.jpg'
    I=cv2.imread(filename1)
    I=redim(I)
    nl,nc=rgb2gray(I).shape
    mat_image=np.zeros((nl,nc,nbr_im))
    
    for i in range(nbr_im):
        if(i<nbr_im/3):
            I2=cv2.imread(pathn+'N'+str(i+1)+'.jpg')
            I2=redim(I2)
            mat_image[:,:,i]=Laplacian(I2)
        elif(i<2*nbr_im/3):
            I2=cv2.imread(pathy+'B'+str(i-int(nbr_im/3)+1)+'.jpg')
            I2=redim(I2)
            mat_image[:,:,i]=Laplacian(I2)
        elif(i<nbr_im):
            I2=cv2.imread(pathy+'M'+str(i-int(2*nbr_im/3)+1)+'.jpg')
            I2=redim(I2)
            mat_image[:,:,i]=Laplacian(I2)
    
    return mat_image

Stockage des descripteurs

In [ ]:
def stocker_desc(mat_image,arr_desc):
    nl,nc,nbr_img=mat_image.shape
    ndesc=arr_desc(mat_image[:,:,0]).shape[0]
    desc=np.zeros((nbr_img,ndesc))
    for i in range(nbr_img):
        desc[i]=arr_desc(mat_image[:,:,i])
    return desc

**Matrices des Images**

In [ ]:
nbr_im=300
nbr_testim=300 #nbr_im=nbr_testim

In [ ]:
mat_image_train=stocker_images_train(nbr_im)
mat_image_train.shape

In [ ]:
mat_lap_train=stocker_images_train_form(nbr_im)
mat_lap_train.shape

**Matrices d'Histogrammes**

In [ ]:
mat_hist_train=stocker_desc(mat_image_train,histo)
mat_hist_train.shape

**Matrices de forme**

In [ ]:
mat_hist_train_form=stocker_desc(mat_lap_train,histo)
mat_hist_train_form.shape

**Matrices de Textures**

In [ ]:
mat_tex_train=stocker_desc(mat_image_train,texture)
mat_tex_train.shape

**Matrices de Textures sur GLCM**

In [ ]:
def stocker_descGLCM(mat_image):
    nl,nc,nbr_img=mat_image.shape
    desc=np.zeros((nbr_img,5))
    for i in range(nbr_img):
        desc[i]=texture(GLCM(mat_image[:,:,i]))
    return desc

In [ ]:
mat_glcm_train=stocker_descGLCM(mat_image_train)
mat_glcm_train.shape

**Matrices des Distances Euclidiennes**

In [ ]:
def stocker_dist(mat_desc_train,mat_desc_test,nbr_img):
    distance_mat=np.zeros((nbr_img))
    for i in range(0,nbr_img):
        distance_mat[i] = distance(mat_desc_train[i],mat_desc_test)
    return distance_mat

**Matrices des Distances de Manhattan**

In [ ]:
def stocker_distM(mat_desc_train,mat_desc_test,nbr_img):
    distance_mat=np.zeros((nbr_img))
    for i in range(0,nbr_img):
        distance_mat[i] = distanceMan(mat_desc_train[i],mat_desc_test)
    return distance_mat

**Prediction measurs**

In [ ]:
def measurs(mat_train,img_req,nbr=300):
    mat_disE=stocker_dist(mat_train,img_req,nbr)
    mat_disM=stocker_distM(mat_train,img_req,nbr)
    mat_indexE=np.argsort(mat_disE)
    mat_indexM=np.argsort(mat_disM)
    K_proche=10
    be=0;me=0;ne=0
    bm=0;mm=0;nm=0
    a=5;b=5
    for i in range(K_proche):
        if mat_indexE[i]<nbr/3:
            ne+=10
        elif mat_indexE[i]<(2*nbr/3):
            be+=10
        elif mat_indexE[i]<nbr:
            me+=10

        
    for i in range(K_proche):
        if mat_indexM[i]<nbr/3:
            nm+=10
        elif mat_indexM[i]<(2*nbr/3):
            bm+=10
        elif mat_indexM[i]<nbr:
            mm+=10

    return [ne,be,me,nm,bm,mm]

**Import Data**

In [ ]:
brain_df=pd.read_csv('data_mask.csv')
brain_df_mask = brain_df[brain_df['mask'] == 1]
brain_df_mask = brain_df_mask.drop(columns = ['patient_id'])

liste_path=[]
liste_mask=[]

for i,j in enumerate(brain_df_mask.image_path):
    a='Dataset/Mask/'+j
    liste_path.append(a)

for k,l in enumerate(brain_df_mask.mask_path):
    b='Dataset/Mask/'+l
    liste_mask.append(b)

**Prepare Data Test**

In [ ]:
from tqdm import tqdm
from skimage.transform import resize
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
print('Getting and resizing training images ... ')
X_train = np.zeros((6, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
        
# Re-sizing our training images to 128 x 128
# Note sys.stdout prints info that can be cleared unlike print.
# Using TQDM allows us to create progress bars
sys.stdout.flush()

for i in range(5):
    path=liste_path[i]
    path1=liste_mask[i]
    img = plt.imread(path)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[i] = img

**Import Segmentation Model**

In [ ]:
def my_iou_metric(label, pred):
    metric_value = tf.py_func(iou_metric_batch, [label, pred], tf.float32)     
    return metric_value

model = load_model('nuclei_finder_unet_1.h5', 
                   custom_objects={'my_iou_metric': my_iou_metric})

**Segmentation**

In [ ]:
from skimage.transform import resize

def segmentation(path,name):
    image='static/img/' +name
    mask='static/img/' + 'mask_'+name 
    img = plt.imread(path,0)
    img = resize(img, (128, 128), mode='constant', preserve_range=True)
    X_train[5]=img
    preds_test = model.predict(X_train, verbose=1)
    preds_test_t = (preds_test > 0.5).astype(np.uint8)
    plt.title('Image Original')
    plt.imshow(X_train[5])
    plt.savefig(image)
    plt.title('Segmentation')
    plt.imshow(np.squeeze(preds_test_t[5]))
    plt.savefig(mask)
    
    return [image,mask]

**App Web**

In [ ]:
app = Flask(__name__)
UPLOAD_FOLDER = 'static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
'''ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS'''


@app.route('/')
def home():
    return render_template('index.html',)


@app.route('/upload', methods=['POST','GET'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        filename1='static/uploads/' + filename
        I=redim(rgb2gray(plt.imread(filename1)))
        histogramme=histo(I)
        I2=redim(cv2.imread(filename1))
        form=histo(Laplacian(I2))
        textur=texture(I)
        glcm=texture(GLCM(I))
        eval_params_histo=measurs(mat_hist_train,histogramme)
        eval_params_form=measurs(mat_hist_train_form,form)
        eval_params_texture=measurs(mat_tex_train,textur)
        eval_params_glcm=measurs(mat_glcm_train,glcm)
        image_url=segmentation(filename1,filename)
        return jsonify(eval_params_histo,eval_params_form,eval_params_texture,eval_params_glcm,image_url)
if __name__ == '__main__':
    app.run()